In [2]:
import numpy as np
# coords = np.array([[0,0], [1,0], [3,5], [10,20]])

In [3]:
# converts coordinate configuration into angles configuration
def coords2angle(coords):   
    thetas = []
    lengths = []
    for ind in range(len(coords)-1):
        edge = coords[ind+1] - coords[ind]
        length = np.linalg.norm(edge)
        lengths.append(length)
        theta = np.arccos(edge[0]/length)
        if edge[1]<0:
            theta+=np.pi
        thetas.append(theta)
    return thetas, lengths

In [4]:
# converts angles configuration back to coordinate configuration
import cmath
def angle2coords(angles, lengths):
    coords = [[0,0]]
    for ind in range(len(angles)):
        # Get the displacement of the new coord as a complex number
        coord = lengths[ind] * np.exp(complex(0, angles[ind]))
        # Convert the complex number to a coord and offset it with the last coord
        coord = [coords[ind][0] + np.real(coord), coords[ind][1] + np.imag(coord)]
        coords.append(coord)
    return coords

In [60]:
coords.size()

TypeError: 'int' object is not callable

In [80]:
def gradient_term(wx, wy, vx, vy, ux, uy):
    g = np.array([[(-(-ux + wx)/sqrt((ux - wx)**2 + (uy - wy)**2) - (-vx + wx)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2,
             (-(-uy + wy)/sqrt((ux - wx)**2 + (uy - wy)**2) - (-vy + wy)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2],
             [((-ux + vx)/sqrt((ux - vx)**2 + (uy - vy)**2) - (vx - wx)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2,
             ((-uy + vy)/sqrt((ux - vx)**2 + (uy - vy)**2) - (vy - wy)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2],
             [((ux - vx)/sqrt((ux - vx)**2 + (uy - vy)**2) - (ux - wx)/sqrt((ux - wx)**2 + (uy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2,
             ((uy - vy)/sqrt((ux - vx)**2 + (uy - vy)**2) - (uy - wy)/sqrt((ux - wx)**2 + (uy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2]])
    return g

In [90]:
# computes `ENERGY` of a configuration and returns the gradient as well :)
def energy(coords):
    E = 0
    G = np.zeros(coords.shape)
    # Iterate through edges u,v
    for u_ind in range(len(coords)-1):
        u = coords[u_ind]
        v = coords[u_ind+1]
        edge = v - u
        # Iterate through vertices w not on the edge
        for w_ind in range(len(coords)):
            if w_ind==u_ind or w_ind==u_ind+1:
                continue
            w = coords[w_ind]
            # Add to the energy
            E+=1/(np.linalg.norm(u-w)+np.linalg.norm(v-w)-np.linalg.norm(u-v))**2
            # Add to the gradient
            g = gradient_term(w[0], w[1], v[0], v[1], u[0], u[1])
            G[w_ind, :] = G[w_ind, :] + g[0]
            G[u_ind + 1, :] = G[u_ind + 1, :] + g[1]
            G[u_ind, :] = G[u_ind, :] + g[2]
    return E,G 

In [94]:
def gradient_descent(coords):
    stepsize = 0.01
    e, gradient = energy(coords)
    new_coords = coords - stepsize * gradient
    return new_coords, gradient

i = 0
while True:
    tolerance = 0.01
    print(i)
    coords, gradient = gradient_descent(coords)
    i += 1
    print(i)
    if np.linalg.norm(gradient) < tolerance:
        break

0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152
15

KeyboardInterrupt: 

In [92]:
energy(coords)

(15601238041.935246,
 array([[-5.42762268e+00,  1.10262004e+01],
        [-8.08146702e+01, -1.09781810e+00],
        [-1.68827191e+01, -6.55475326e+00],
        [-5.15880876e+00, -1.28789774e+01],
        [ 9.20198279e+00,  1.02549746e+01],
        [-1.43261277e+01,  2.39458098e+01],
        [-1.37608985e+02, -2.11066819e+02],
        [ 2.78045931e+07,  6.60470252e+07],
        [-1.44665094e+07, -3.42579192e+07],
        [-1.33378327e+07, -3.17889197e+07]]))

In [12]:
coords2angle(coords)

([0.0,
  1.5707963267948966,
  3.141592653589793,
  4.71238898038469,
  0.0,
  1.5707963267948966,
  3.141592653589793,
  4.71238898038469,
  0.0],
 [1.0, 1.0, 1.0, 0.75, 0.75, 0.5, 0.5, 0.25, 0.25])

In [65]:
coords = np.array([[0.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 0.25],
       [0.75, 0.25],
       [0.75, 0.75],
       [0.25, 0.75],
       [0.25, 0.5 ],
       [0.5 , 0.5 ]])
np.zeros(coords.shape)

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [14]:
print(coords)
angles, lengths = coords2angle(coords)
angle2coords(angles, lengths)

[[0.   0.  ]
 [1.   0.  ]
 [1.   1.  ]
 [0.   1.  ]
 [0.   0.25]
 [0.75 0.25]
 [0.75 0.75]
 [0.25 0.75]
 [0.25 0.5 ]
 [0.5  0.5 ]]


[[0, 0],
 [1.0, 0.0],
 [1.0, 1.0],
 [0.0, 1.0000000000000002],
 [-1.3777276490407724e-16, 0.2500000000000002],
 [0.7499999999999999, 0.2500000000000002],
 [0.7499999999999999, 0.7500000000000002],
 [0.2499999999999999, 0.7500000000000003],
 [0.24999999999999983, 0.5000000000000003],
 [0.49999999999999983, 0.5000000000000003]]

In [29]:
from sympy import *
from sympy.vector import gradient
init_printing()

In [31]:
wx, wy, vx, vy, ux, uy = symbols('wx wy vx vy ux uy ', real = True)
w = Matrix([wx, wy])
v = Matrix([vx, vy])
u = Matrix([ux, uy])
expr = (w-v).norm(2) + (w-u).norm(2) - (v-u).norm(2)
exp = 1/expr
# gradient(exp)
l = [wx, wy, vx, vy, ux, uy ]
for var in l:
    print(diff(exp,var))

(-(-ux + wx)/sqrt((ux - wx)**2 + (uy - wy)**2) - (-vx + wx)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2
(-(-uy + wy)/sqrt((ux - wx)**2 + (uy - wy)**2) - (-vy + wy)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2
((-ux + vx)/sqrt((ux - vx)**2 + (uy - vy)**2) - (vx - wx)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2
((-uy + vy)/sqrt((ux - vx)**2 + (uy - vy)**2) - (vy - wy)/sqrt((vx - wx)**2 + (vy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx - wx)**2 + (vy - wy)**2))**2
((ux - vx)/sqrt((ux - vx)**2 + (uy - vy)**2) - (ux - wx)/sqrt((ux - wx)**2 + (uy - wy)**2))/(-sqrt((ux - vx)**2 + (uy - vy)**2) + sqrt((ux - wx)**2 + (uy - wy)**2) + sqrt((vx -